In [8]:
import math

class Node:
    def __init__(self, x, y, _id=None):
        self.x = x
        self.y = y
        self.dis = 0
        self.id = _id
        self.degree = 0

    def update(self, other):
        tmp_x = self.x - other.x
        tmp_y = self.y - other.y
        self.dis = math.sqrt(tmp_x * tmp_x + tmp_y * tmp_y)
        self.degree = math.atan2(tmp_y, tmp_x)
        if self.degree < 0:
            self.degree += 2 * math.pi

    def print(self):
        print(f"{self.x} {self.y}")

class Edge:
    def __init__(self, start, end):
        self.start = start
        self.end = end

    def print(self):
        print("Line segment endpoints:")
        self.start.print()
        self.end.print()

class Graph:
    def __init__(self):
        self.node_set = []
        self.edge_set = []

    def push_node(self, p):
        self.node_set.append(p)

    def add_node(self, nodes):
        self.node_set.extend(nodes)

    def add_edge(self, edges):
        self.edge_set.extend(edges)

    def add_edge_obj(self, tmp):
        new_edge = Edge(Node(tmp.start.x, tmp.start.y), Node(tmp.end.x, tmp.end.y))
        self.edge_set.append(new_edge)

class Polygon:
    def __init__(self):
        self.node_num = 0
        self.node_set = []
        self.edge_set = []
        self.max_node = None
        self.min_node = None

    def push_node(self, p):
        tmp = Node(p.x, p.y, p.id)
        self.node_set.append(tmp)
        self.node_num += 1
        if self.max_node is None:
            self.max_node = tmp
        if self.min_node is None:
            self.min_node = tmp
        if self.max_node.degree - tmp.degree < 0:
            self.max_node = tmp
        if self.min_node.degree - tmp.degree > 0:
            self.min_node = tmp
        if self.node_num > 1:
            self.add_edge(tmp, self.node_set[self.node_num - 2])

    def add_edge(self, p1, p2):
        tmp_edge = Edge(Node(p1.x, p1.y), Node(p2.x, p2.y))
        self.edge_set.append(tmp_edge)

    def set_edge(self):
        for i in range(len(self.node_set)):
            p1 = self.node_set[i]
            p2 = self.node_set[(i + 1) % len(self.node_set)]
            tmp_edge = Edge(Node(p1.x, p1.y), Node(p2.x, p2.y))
            self.edge_set.append(tmp_edge)

def get_x(e, p):
    x1 = e.start.x - p.x
    y1 = e.start.y - p.y
    x2 = e.end.x - p.x
    y2 = e.end.y - p.y
    if y1 * y2 < 0:
        return (1.0 * (x1 - x2) / (y1 - y2) * (-y2) + x2)
    elif y1 == 0:
        return x1
    elif y2 == 0:
        return x2
    else:
        return -1

def cmp(a, b):
    if a.degree != b.degree:
        return a.degree < b.degree
    return a.dis < b.dis

def visible_node(e, p, o):
    x_1 = o.x
    y_1 = o.y
    x_2 = e.start.x
    y_2 = e.start.y
    x_3 = e.end.x
    y_3 = e.end.y
    tmp_1 = x_1 * y_2 + x_2 * y_3 + x_3 * y_1 - x_1 * y_3 - x_2 * y_1 - x_3 * y_2
    x_1 = p.x
    y_1 = p.y
    tmp_2 = x_1 * y_2 + x_2 * y_3 + x_3 * y_1 - x_1 * y_3 - x_2 * y_1 - x_3 * y_2
    if tmp_1 * tmp_2 < 0:
        return False
    else:
        return True

def visible_check(p, polygon_set):
    vis_node = []
    all_node = []
    all_edge = []
    node_to_edge = {}
    node_to_polygon = {}
    scan_line = []

    for polygon in polygon_set:
        tmp = polygon.node_set
        all_node.extend(tmp)
        tmp_edge = polygon.edge_set
        all_edge.extend(tmp_edge)
        for i in range(len(tmp)):
            tmp_node = tmp[i]
            tmp_edge_1 = tmp_edge[i]
            tmp_edge_2 = tmp_edge[(i - 1 + len(tmp)) % len(tmp)]
            node_to_edge[tmp_node] = (tmp_edge_1, tmp_edge_2)
            node_to_polygon[tmp_node] = i

    for tmp in all_node:
        tmp.update(p)

    all_node.sort(key=lambda x: (x.degree, x.dis))

    for i in range(len(all_edge)):
        tmp = all_edge[i]
        node_1 = tmp.start
        node_2 = tmp.end
        x1 = node_1.x - p.x
        y1 = node_1.y - p.y
        x2 = node_2.x - p.x
        y2 = node_2.y - p.y
        if y1 * y2 < 0:
            if (1.0 * (x1 - x2) / (y1 - y2) * (-y2) + x2) >= 0:
                scan_line.append(tmp)

    for i in range(len(all_node)):
        tmp_node = all_node[i]
        index = 0
        if node_to_polygon[p] == node_to_polygon[tmp_node]:
            index = 0
        else:
            if not scan_line:
                vis_node.append(tmp_node)
            else:
                top = scan_line[0]
                if visible_node(top, tmp_node, p):
                    vis_node.append(tmp_node)
                    index = 0
                else:
                    index = 1
                    while index < len(scan_line) and not visible_node(scan_line[index], tmp_node, p):
                        index += 1

        tmp_edge_1, tmp_edge_2 = node_to_edge[tmp_node]
        if tmp_edge_1.start.id != tmp_node.id:
            tmp_edge_1.start, tmp_edge_1.end = tmp_edge_1.end, tmp_edge_1.start
        if tmp_edge_2.start.id != tmp_node.id:
            tmp_edge_2.start, tmp_edge_2.end = tmp_edge_2.end, tmp_edge_2.start

        x_1, y_1 = p.x, p.y
        x_2, y_2 = tmp_node.x, tmp_node.y
        x_3, y_3 = tmp_edge_1.end.x, tmp_edge_1.end.y
        test_edge_1 = x_1 * y_2 + x_2 * y_3 + x_3 * y_1 - x_1 * y_3 - x_2 * y_1 - x_3 * y_2

        x_3, y_3 = tmp_edge_2.end.x, tmp_edge_2.end.y
        test_edge_2 = x_1 * y_2 + x_2 * y_3 + x_3 * y_1 - x_1 * y_3 - x_2 * y_1 - x_3 * y_2

        x_1, y_1 = tmp_edge_1.end.x, tmp_edge_1.end.y
        test_order = x_1 * y_2 + x_2 * y_3 + x_3 * y_1 - x_1 * y_3 - x_2 * y_1 - x_3 * y_2

        if test_edge_1 > 0 and test_edge_2 > 0:
            if test_order > 0:
                scan_line.insert(index, tmp_edge_2)
                scan_line.insert(index, tmp_edge_1)
            else:
                scan_line.insert(index, tmp_edge_1)
                scan_line.insert(index, tmp_edge_2)
        else:
            if test_edge_1 < 0:
                for j in range(index, len(scan_line)):
                    if (scan_line[j].start.x == tmp_edge_1.start.x and scan_line[j].start.y == tmp_edge_1.start.y) or (
                            scan_line[j].end.x == tmp_edge_1.start.x and scan_line[j].end.y == tmp_edge_1.start.y):
                        scan_line.pop(j)
                        break
            if test_edge_2 < 0:
                for j in range(index, len(scan_line)):
                    if (scan_line[j].start.x == tmp_edge_2.start.x and scan_line[j].start.y == tmp_edge_2.start.y) or (
                            scan_line[j].end.x == tmp_edge_2.start.x and scan_line[j].end.y == tmp_edge_2.start.y):
                        scan_line.pop(j)
                        break
            if test_edge_1 > 0:
                scan_line.insert(index, tmp_edge_1)
            if test_edge_2 > 0:
                scan_line.insert(index, tmp_edge_2)

    return vis_node

def visible_map(g, polygon_set):
    node_set = g.node_set
    for p in node_set:
        vis_node = visible_check(p, polygon_set)
        for tmp_node in vis_node:
            tmp_edge = Edge(Node(p.x, p.y), Node(tmp_node.x, tmp_node.y))
            g.add_edge_obj(tmp_edge)

# Create start node
start_node = Node(0, 0)
polygon_1 = Polygon()
polygon_1.push_node(Node(1, 2, 1))
polygon_1.push_node(Node(1, 4, 2))
polygon_1.push_node(Node(3, 5, 3))
polygon_1.push_node(Node(4, 1, 4))
polygon_1.push_node(Node(3, 0, 5))
polygon_1.set_edge()

polygon_2 = Polygon()
polygon_2.push_node(Node(2, 0, 1))
polygon_2.push_node(Node(3, -1, 2))
polygon_2.push_node(Node(5, 1, 3))
polygon_2.push_node(Node(4, -2, 4))
polygon_2.push_node(Node(2, -4, 5))
polygon_2.push_node(Node(1, -1, 6))
polygon_2.set_edge()

g = Graph()
g.add_node(polygon_1.node_set)
g.add_node(polygon_2.node_set)
g.add_edge(polygon_1.edge_set)
g.add_edge(polygon_2.edge_set)

polygon_set = [polygon_1, polygon_2]

# visible_map(g, polygon_set)
# for edge in g.edge_set:
#     edge.print()

# p=Node(0,0)
visible_check(start_node,polygon_set)


KeyError: <__main__.Node object at 0x000002ABE192B0D0>